In [1]:
library(lavaan)     # CFA/ESEM/SEM
library(semTools)   # invariance helpers
library(dplyr)
library(readr)

dat <- readRDS("data/processed/analysis_phase1_ordered.rds")

# Item sets
dass_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A",
                "dQ8S","dQ9A","dQ10D","dQ11S","dQ12S","dQ13D",
                "dQ14S","dQ15A","dQ16D","dQ17D","dQ18S","dQ19A",
                "dQ20A","dQ21D")

brs_items_scored <- c("rQ1","rQ3","rQ5","rQ2_r","rQ4_r","rQ6_r")  # scored orientation

# Ensure ordered type for lavaan
dat[dass_items] <- lapply(dat[dass_items], function(x) factor(as.integer(as.character(x)), levels = 0:3, ordered = TRUE))
dat[brs_items_scored] <- lapply(dat[brs_items_scored], function(x) factor(as.integer(as.character(x)), levels = 1:5, ordered = TRUE))


This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.
 
###############################################################################
This is semTools 0.5-7
All users of R (or SEM) are invited to submit functions or ideas for functions.
###############################################################################

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘readr’

The following object is masked from ‘package:semTools’:

    clipboard



In [7]:
# Assign items by domain based on the DASS-21 key
dep_items <- c("dQ3D","dQ5D","dQ10D","dQ13D","dQ16D","dQ17D","dQ21D")
anx_items <- c("dQ2A","dQ4A","dQ7A","dQ9A","dQ15A","dQ19A","dQ20A")
str_items <- c("dQ1S","dQ6S","dQ8S","dQ11S","dQ12S","dQ14S","dQ18S")

ctrl <- list(iter.max = 20000, rel.tol = 1e-6)

fit_cfa <- cfa(model_cfa, data = dat,
               ordered = dass_items,
               estimator = "WLSMV",
               parameterization = "theta",
               std.lv = FALSE,
               control = ctrl)
summary(fit_cfa, fit.measures = TRUE, standardized = TRUE)


In [10]:
# ESEM with 3 factors for DASS items
model_esem <- '
efa("dass_block")*F1 + efa("dass_block")*F2 + efa("dass_block")*F3 =~
  dQ1S + dQ2A + dQ3D + dQ4A + dQ5D + dQ6S + dQ7A + dQ8S + dQ9A +
  dQ10D + dQ11S + dQ12S + dQ13D + dQ14S + dQ15A + dQ16D + dQ17D +
  dQ18S + dQ19A + dQ20A + dQ21D
'

fit_esem <- cfa(model_esem, data = dat,
                ordered = dass_items,
                estimator = "WLSMV",
                parameterization = "theta",
                std.lv = TRUE,
                rotation = "oblimin",
                control = ctrl)
summary(fit_esem, fit.measures = TRUE, standardized = TRUE)


In [11]:
# Bifactor CFA (general + group factors), orthogonal group factors typical
model_bifactor <- '
G =~ dQ1S + dQ2A + dQ3D + dQ4A + dQ5D + dQ6S + dQ7A + dQ8S + dQ9A +
      dQ10D + dQ11S + dQ12S + dQ13D + dQ14S + dQ15A + dQ16D + dQ17D +
      dQ18S + dQ19A + dQ20A + dQ21D
Dep =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
Anx =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
Str =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S

# Orthogonality constraints (typical bifactor)
G ~~ 0*Dep
G ~~ 0*Anx
G ~~ 0*Str
Dep ~~ 0*Anx
Dep ~~ 0*Str
Anx ~~ 0*Str
'

fit_bif <- cfa(model_bifactor, data = dat,
               ordered = dass_items,
               estimator = "WLSMV",
               parameterization = "theta",
               std.lv = TRUE,
               control = ctrl)
summary(fit_bif, fit.measures = TRUE, standardized = TRUE)



In [12]:
fitMeasures(fit_cfa, c("cfi","tli","rmsea","srmr"))
fitMeasures(fit_esem, c("cfi","tli","rmsea","srmr"))
fitMeasures(fit_bif, c("cfi","tli","rmsea","srmr"))


In [13]:
model_brs <- '
BRS =~ rQ1 + rQ3 + rQ5 + rQ2_r + rQ4_r + rQ6_r
'
fit_brs <- cfa(model_brs, data = dat, ordered = brs_items_scored, estimator = "WLSMV")
summary(fit_brs, fit.measures = TRUE, standardized = TRUE)


In [14]:
# Example using 3-factor DASS; swap in bifactor syntax if selected
model_joint <- '
Dep =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
Anx =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
Str =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S
BRS =~ rQ1 + rQ3 + rQ5 + rQ2_r + rQ4_r + rQ6_r
'
fit_joint <- cfa(model_joint, data = dat, ordered = c(dass_items, brs_items_scored),
                 estimator = "WLSMV")
summary(fit_joint, fit.measures = TRUE, standardized = TRUE)


In [19]:
library(haven); library(dplyr)

# Start from your Phase 2 data frame 'dat'
dat <- readRDS("data/processed/analysis_phase1_ordered.rds")

# Convert haven_labelled -> factor with readable levels
dat <- dat %>%
  mutate(
    gender        = haven::as_factor(gender, levels = "labels"),
    academic_year = haven::as_factor(academic_year, levels = "labels"),
    residence     = haven::as_factor(residence, levels = "labels")
  )

# Drop rows with missing group labels for the test at hand
dat_gender  <- dplyr::filter(dat, !is.na(gender))
dat_acadyr  <- dplyr::filter(dat, !is.na(academic_year))


In [20]:
library(lavaan)
library(semTools)

# DASS 3-factor model (replace with your chosen model)
model_cfa <- '
Dep =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
Anx =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
Str =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S
'

dass_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S",
                "dQ9A","dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A",
                "dQ16D","dQ17D","dQ18S","dQ19A","dQ20A","dQ21D")

# CONFIGURAL (no equality constraints)
cfg_gender <- measEq.syntax(
  configural.model = model_cfa,
  data  = dat_gender,
  ordered = dass_items,
  parameterization = "theta",
  ID.fac = "std.lv",
  ID.cat = "Wu.Estabrook.2016",
  group = "gender",
  group.equal = c(),
  estimator = "WLSMV",
  return.fit = TRUE
)

# THRESHOLD INVARIANCE (equal thresholds across groups)
thr_gender <- measEq.syntax(
  configural.model = model_cfa,
  data  = dat_gender,
  ordered = dass_items,
  parameterization = "theta",
  ID.fac = "std.lv",
  ID.cat = "Wu.Estabrook.2016",
  group = "gender",
  group.equal = c("thresholds"),
  estimator = "WLSMV",
  return.fit = TRUE
)

# THRESHOLDS + LOADINGS (often the key level for ordinal data)
met_gender <- measEq.syntax(
  configural.model = model_cfa,
  data  = dat_gender,
  ordered = dass_items,
  parameterization = "theta",
  ID.fac = "std.lv",
  ID.cat = "Wu.Estabrook.2016",
  group = "gender",
  group.equal = c("thresholds","loadings"),
  estimator = "WLSMV",
  return.fit = TRUE
)

# Compare fits (ΔCFI, ΔRMSEA) and review summaries
semTools::compareFit(cfg_gender, thr_gender, met_gender)
summary(cfg_gender, fit.measures = TRUE, standardized = TRUE)
summary(thr_gender, fit.measures = TRUE, standardized = TRUE)
summary(met_gender, fit.measures = TRUE, standardized = TRUE)


In [21]:
cfg_year <- measEq.syntax(model_cfa, dat_acadyr, ordered = dass_items,
                          parameterization = "theta", ID.fac = "std.lv",
                          ID.cat = "Wu.Estabrook.2016", group = "academic_year",
                          group.equal = c(), estimator = "WLSMV", return.fit = TRUE)

thr_year <- measEq.syntax(model_cfa, dat_acadyr, ordered = dass_items,
                          parameterization = "theta", ID.fac = "std.lv",
                          ID.cat = "Wu.Estabrook.2016", group = "academic_year",
                          group.equal = c("thresholds"), estimator = "WLSMV",
                          return.fit = TRUE)

met_year <- measEq.syntax(model_cfa, dat_acadyr, ordered = dass_items,
                          parameterization = "theta", ID.fac = "std.lv",
                          ID.cat = "Wu.Estabrook.2016", group = "academic_year",
                          group.equal = c("thresholds","loadings"),
                          estimator = "WLSMV", return.fit = TRUE)

semTools::compareFit(cfg_year, thr_year, met_year)


Warning message:
lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, "cov.lv") to 
   investigate. 
Warning message:
lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, "cov.lv") to 
   investigate. 
Warning messages:
1: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, "cov.lv") to 
   investigate. 
2: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 3; use lavInspect(fit, "cov.lv") to 
   investigate. 


In [23]:
library(dplyr)

# 1) Get the index of rows used in each fit
idx_cfa <- lavInspect(fit_cfa, "case.idx")   # indices of dat rows used in CFA
idx_brs <- lavInspect(fit_brs, "case.idx")   # indices of dat rows used in BRS CFA

# 2) Extract scores
scores_dass <- lavPredict(fit_cfa, method = "EBM", type = "lv")  # matrix with rows = length(idx_cfa)
scores_brs  <- lavPredict(fit_brs,  method = "EBM", type = "lv")

# 3) Build score data frames with the original row indices for reliable merging
df_dass <- data.frame(.row = idx_cfa,
                      DASS_Dep = as.numeric(scores_dass[,"Dep"]),
                      DASS_Anx = as.numeric(scores_dass[,"Anx"]),
                      DASS_Str = as.numeric(scores_dass[,"Str"]))

df_brs  <- data.frame(.row = idx_brs,
                      BRS = as.numeric(scores_brs[,"BRS"]))

# 4) Attach a row index to the full data (keeps order intact)
dat$.row <- seq_len(nrow(dat))

# 5) Left-join scores back to the full dataset by .row
dat_scores <- dat %>%
  left_join(df_dass, by = ".row") %>%
  left_join(df_brs,  by = ".row") %>%
  # Standardize only where scores exist
  mutate(across(c(DASS_Dep, DASS_Anx, DASS_Str, BRS),
                ~ ifelse(is.na(.x), NA_real_, as.numeric(scale(.x)))))

# 6) Save
saveRDS(dat_scores, "data/processed/phase2_factor_scores.rds")
write.csv(dat_scores %>% select(id, DASS_Dep, DASS_Anx, DASS_Str, BRS),
          "data/processed/phase2_factor_scores.csv", row.names = FALSE)


In [24]:
# Compare key fit indices side-by-side
fits <- list(CFA = fit_cfa, ESEM = fit_esem, BIFACTOR = fit_bif)
sapply(fits, function(x) fitMeasures(x, c("cfi","tli","rmsea","srmr","chisq","df")))

# Inspect standardized loadings to judge interpretability
summary(fit_cfa, fit.measures = TRUE, standardized = TRUE)
summary(fit_esem, fit.measures = TRUE, standardized = TRUE)
summary(fit_bif, fit.measures = TRUE, standardized = TRUE)

# Choose one final model object for DASS
fit_dass_final <- fit_esem  # or fit_cfa, or fit_bif, based on your comparison


In [25]:
summary(fit_brs, fit.measures = TRUE, standardized = TRUE)


In [26]:
# Example shown for 3-factor DASS. If using ESEM/bifactor, adapt syntax accordingly.
# For ESEM: estimating a joint model can be heavy; reporting the correlation from a sequential approach is acceptable.

# If your final choice is CFA (Dep/Anx/Str), run joint model:
model_joint <- '
Dep =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
Anx =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
Str =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S
BRS =~ rQ1 + rQ3 + rQ5 + rQ2_r + rQ4_r + rQ6_r
'
fit_joint <- cfa(model_joint, data = dat, ordered = c(dass_items, brs_items_scored),
                 estimator = "WLSMV", parameterization = "theta", std.lv = TRUE)
inspect(fit_joint, "cor.lv")


In [27]:
library(haven); library(dplyr); library(semTools); library(lavaan)

# Load data and ensure ordered indicators exist
dat <- readRDS("data/processed/analysis_phase1_ordered.rds")

# Convert group variables to factors (not haven_labelled)
dat <- dat %>%
  mutate(
    gender        = haven::as_factor(gender, levels = "labels"),
    academic_year = haven::as_factor(academic_year, levels = "labels")
  )

# Drop rows with missing group labels
dat_gender <- dplyr::filter(dat, !is.na(gender))
dat_year   <- dplyr::filter(dat, !is.na(academic_year))

# Model syntax used for invariance must match your final DASS choice.
# Example below uses 3-factor CFA syntax; if your final DASS is ESEM, do invariance on a CFA representation,
# or test invariance at the item-threshold level for a reduced set. Many papers compare CFA structures across groups for comparability.

model_cfa <- '
Dep =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
Anx =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
Str =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S
'
dass_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S",
                "dQ9A","dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A",
                "dQ16D","dQ17D","dQ18S","dQ19A","dQ20A","dQ21D")

# Gender invariance
cfg_gender <- measEq.syntax(
  configural.model = model_cfa,
  data  = dat_gender,
  ordered = dass_items,
  parameterization = "theta",
  ID.fac = "std.lv",
  ID.cat = "Wu.Estabrook.2016",
  group = "gender",
  group.equal = c(),                 # configural
  estimator = "WLSMV",
  return.fit = TRUE
)
thr_gender <- measEq.syntax(
  configural.model = model_cfa, data = dat_gender, ordered = dass_items,
  parameterization = "theta", ID.fac = "std.lv", ID.cat = "Wu.Estabrook.2016",
  group = "gender", group.equal = c("thresholds"),
  estimator = "WLSMV", return.fit = TRUE
)
met_gender <- measEq.syntax(
  configural.model = model_cfa, data = dat_gender, ordered = dass_items,
  parameterization = "theta", ID.fac = "std.lv", ID.cat = "Wu.Estabrook.2016",
  group = "gender", group.equal = c("thresholds","loadings"),
  estimator = "WLSMV", return.fit = TRUE
)

# Compare fits and print summaries
semTools::compareFit(cfg_gender, thr_gender, met_gender)
summary(cfg_gender, fit.measures = TRUE)
summary(thr_gender, fit.measures = TRUE)
summary(met_gender, fit.measures = TRUE)

# Academic year invariance
cfg_year <- measEq.syntax(
  configural.model = model_cfa, data = dat_year, ordered = dass_items,
  parameterization = "theta", ID.fac = "std.lv", ID.cat = "Wu.Estabrook.2016",
  group = "academic_year", group.equal = c(),
  estimator = "WLSMV", return.fit = TRUE
)
thr_year <- measEq.syntax(
  configural.model = model_cfa, data = dat_year, ordered = dass_items,
  parameterization = "theta", ID.fac = "std.lv", ID.cat = "Wu.Estabrook.2016",
  group = "academic_year", group.equal = c("thresholds"),
  estimator = "WLSMV", return.fit = TRUE
)
met_year <- measEq.syntax(
  configural.model = model_cfa, data = dat_year, ordered = dass_items,
  parameterization = "theta", ID.fac = "std.lv", ID.cat = "Wu.Estabrook.2016",
  group = "academic_year", group.equal = c("thresholds","loadings"),
  estimator = "WLSMV", return.fit = TRUE
)

semTools::compareFit(cfg_year, thr_year, met_year)
summary(cfg_year, fit.measures = TRUE)
summary(thr_year, fit.measures = TRUE)
summary(met_year, fit.measures = TRUE)


Warning message:
lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, "cov.lv") to 
   investigate. 
Warning message:
lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, "cov.lv") to 
   investigate. 
Warning messages:
1: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, "cov.lv") to 
   investigate. 
2: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 3; use lavInspect(fit, "cov.lv") to 
   investigate. 


In [28]:
# Choose which DASS fit you will score from
fit_to_score <- fit_cfa  # or fit_esem, or fit_bif, depending on what you selected

# Case indices used in each fit
idx_dass <- lavInspect(fit_to_score, "case.idx")
idx_brs  <- lavInspect(fit_brs,      "case.idx")

# Scores
sc_dass <- lavPredict(fit_to_score, method = "EBM", type = "lv")
sc_brs  <- lavPredict(fit_brs,      method = "EBM", type = "lv")

# Build score frames with the original row index
df_dass <- data.frame(.row = idx_dass, sc_dass, check.names = FALSE)
df_brs  <- data.frame(.row = idx_brs,  sc_brs,  check.names = FALSE)

# Attach row index and merge
dat$.row <- seq_len(nrow(dat))
dat_scores <- dat %>%
  left_join(df_dass, by = ".row") %>%
  left_join(df_brs,  by = ".row")

# Standardize the columns that exist in the chosen fit
score_names <- setdiff(colnames(df_dass), ".row")
score_names <- c(score_names, setdiff(colnames(df_brs), ".row"))
dat_scores[score_names] <- lapply(dat_scores[score_names], function(x) {
  ifelse(is.na(x), NA_real_, as.numeric(scale(x)))
})

# Rename for clarity depending on model
if (identical(fit_to_score, fit_cfa)) {
  names(dat_scores)[names(dat_scores) %in% c("Dep","Anx","Str")] <- c("DASS_Dep","DASS_Anx","DASS_Str")
}
if ("G" %in% names(dat_scores)) {
  names(dat_scores)[names(dat_scores) == "G"] <- "DASS_G"
}

if ("BRS" %in% names(dat_scores)) {
  # already labeled as BRS
} else if ("BRS" %in% colnames(sc_brs)) {
  # ok
} else {
  # if factor is named "BRS" differently, rename accordingly
}

saveRDS(dat_scores, "data/processed/phase2_factor_scores.rds")
write.csv(dat_scores[, c("id", intersect(c("DASS_Dep","DASS_Anx","DASS_Str","DASS_G","BRS"),
                                         names(dat_scores)))],
          "data/processed/phase2_factor_scores.csv", row.names = FALSE)


In [30]:
library(dplyr)
library(lavaan)

# Helper that always returns a one-row tibble with fixed numeric columns
fi <- function(fit) {
  if (is.null(fit) || !inherits(fit, "lavaan") || !lavInspect(fit, "converged")) {
    tibble::tibble(
      cfi = NA_real_, tli = NA_real_, rmsea = NA_real_, srmr = NA_real_,
      chisq = NA_real_, df = NA_real_
    )
  } else {
    v <- fitMeasures(fit, c("cfi","tli","rmsea","srmr","chisq","df"))
    tibble::tibble(
      cfi   = unname(v["cfi"]),
      tli   = unname(v["tli"]),
      rmsea = unname(v["rmsea"]),
      srmr  = unname(v["srmr"]),
      chisq = unname(v["chisq"]),
      df    = unname(v["df"])
    )
  }
}

# Build the comparison table with consistent shapes and an ID column
fits_list <- list(CFA = fi(fit_cfa),
                  ESEM = fi(fit_esem),
                  BIFACTOR = fi(fit_bif))

fits_tbl <- dplyr::bind_rows(fits_list, .id = "Model")

# Save
write.csv(fits_tbl, "outputs/diagnostics/dass_model_fit_comparison.csv", row.names = FALSE)


In [ ]:
library(ggplot2); library(tidyr); library(dplyr)

# DASS items distribution
dass_long <- dat |>
  select(all_of(dass_items)) |>
  mutate(across(everything(), ~ factor(as.integer(as.character(.)), levels = 0:3))) |>
  pivot_longer(everything(), names_to = "Item", values_to = "Resp")

p_dass_dist <- ggplot(dass_long, aes(Resp)) +
  geom_bar(fill = "#3B82F6") +
  facet_wrap(~ Item, ncol = 7, scales = "free_y") +
  theme_minimal(base_size = 10) +
  labs(title = "DASS-21 item distributions (0–3)", x = "Category", y = "Count")
ggsave("outputs/diagnostics/plot_dass_item_distributions.png", p_dass_dist, width = 12, height = 7, dpi = 200)

# BRS items distribution
brs_long <- dat |>
  select(all_of(brs_items_scored)) |>
  mutate(across(everything(), ~ factor(as.integer(as.character(.)), levels = 1:5))) |>
  pivot_longer(everything(), names_to = "Item", values_to = "Resp")

p_brs_dist <- ggplot(brs_long, aes(Resp)) +
  geom_bar(fill = "#10B981") +
  facet_wrap(~ Item, ncol = 6, scales = "free_y") +
  theme_minimal(base_size = 10) +
  labs(title = "BRS item distributions (1–5)", x = "Category", y = "Count")
ggsave("outputs/diagnostics/plot_brs_item_distributions.png", p_brs_dist, width = 10, height = 5, dpi = 200)


In [ ]:
library(psych); library(reshape2)

# DASS polychoric
dass_df <- dat |> select(all_of(dass_items)) |> mutate(across(everything(), ~ as.ordered(.)))
dass_poly <- psych::polychoric(dass_df)$rho
melt_dass <- reshape2::melt(dass_poly, varnames = c("i","j"), value.name = "r")

p_poly_dass <- ggplot(melt_dass, aes(i, j, fill = r)) +
  geom_tile() + scale_fill_gradient2(low = "#ef4444", mid = "white", high = "#0ea5e9", midpoint = 0) +
  theme_minimal(base_size = 9) + theme(axis.text.x = element_text(angle = 90, vjust = .5)) +
  labs(title = "Polychoric correlations: DASS-21", x = "", y = "", fill = "r")
ggsave("outputs/diagnostics/plot_polychoric_dass.png", p_poly_dass, width = 8, height = 7, dpi = 200)

# BRS polychoric (scored orientation)
brs_df <- dat |> select(all_of(brs_items_scored)) |> mutate(across(everything(), ~ as.ordered(.)))
brs_poly <- psych::polychoric(brs_df)$rho
melt_brs <- reshape2::melt(brs_poly, varnames = c("i","j"), value.name = "r")

p_poly_brs <- ggplot(melt_brs, aes(i, j, fill = r)) +
  geom_tile() + scale_fill_gradient2(low = "#ef4444", mid = "white", high = "#10b981", midpoint = 0) +
  theme_minimal(base_size = 9) + theme(axis.text.x = element_text(angle = 90, vjust = .5)) +
  labs(title = "Polychoric correlations: BRS", x = "", y = "", fill = "r")
ggsave("outputs/diagnostics/plot_polychoric_brs.png", p_poly_brs, width = 6, height = 5, dpi = 200)


In [33]:
# Use your chosen DASS model object
fit_dass_final <- fit_esem  # or fit_cfa or fit_bif

resid_dass <- lavResiduals(fit_dass_final, type = "cor")$cov  # residual correlations
melt_resid <- reshape2::melt(resid_dass, varnames = c("i","j"), value.name = "r")

p_resid <- ggplot(melt_resid, aes(i, j, fill = r)) +
  geom_tile() + scale_fill_gradient2(low = "#dc2626", mid = "white", high = "#16a34a", midpoint = 0) +
  theme_minimal(base_size = 9) + theme(axis.text.x = element_text(angle = 90, vjust = .5)) +
  labs(title = "Residual correlations (final DASS model)", x = "", y = "", fill = "resid r")
ggsave("outputs/diagnostics/plot_residual_cor_final_dass.png", p_resid, width = 8, height = 7, dpi = 200)


In [34]:
library(dplyr)

# Extract standardized solutions for final DASS model
std_final <- standardizedSolution(fit_dass_final)
load_tbl <- std_final |>
  filter(op == "=~") |>
  transmute(Factor = lhs, Item = rhs, Loading = est.std)

p_load <- ggplot(load_tbl, aes(x = Loading, y = reorder(Item, Loading), color = Factor)) +
  geom_point(size = 2) + geom_vline(xintercept = 0.30, linetype = 2, color = "gray50") +
  theme_minimal(base_size = 10) +
  labs(title = "Standardized loadings (final DASS model)", x = "Loading", y = "Item")
ggsave("outputs/diagnostics/plot_loadings_final_dass.png", p_load, width = 8, height = 7, dpi = 200)

# BRS loadings
std_brs <- standardizedSolution(fit_brs) |>
  filter(op == "=~") |>
  transmute(Item = rhs, Loading = est.std)

p_brs_load <- ggplot(std_brs, aes(x = Loading, y = reorder(Item, Loading))) +
  geom_point(color = "#10B981", size = 2) + geom_vline(xintercept = 0.30, linetype = 2, color = "gray50") +
  theme_minimal(base_size = 10) +
  labs(title = "Standardized loadings (BRS one-factor)", x = "Loading", y = "Item")
ggsave("outputs/diagnostics/plot_loadings_brs.png", p_brs_load, width = 6, height = 4, dpi = 200)


In [35]:
# If you fitted a joint model (fit_joint), do:
# cor_lv <- inspect(fit_joint, "cor.lv")

# Otherwise for final DASS alone:
cor_lv_dass <- inspect(fit_dass_final, "cor.lv")
melt_cor <- reshape2::melt(cor_lv_dass, varnames = c("i","j"), value.name = "r")

p_cor <- ggplot(melt_cor, aes(i, j, fill = r)) +
  geom_tile() + scale_fill_gradient2(low = "#ef4444", mid = "white", high = "#3b82f6", midpoint = 0) +
  geom_text(aes(label = sprintf("%.2f", r)), size = 3) +
  theme_minimal(base_size = 9) + labs(title = "Latent factor correlations (DASS)", x = "", y = "", fill = "r")
ggsave("outputs/diagnostics/plot_factor_correlations_dass.png", p_cor, width = 5, height = 4, dpi = 200)

# For a quick BRS–DASS association (if no joint model), compute Pearson between BRS factor score and a DASS factor score as a sanity check:
# Use your saved dat_scores to visualize relations later.


In [38]:
# Safe fit table (from previous fix)
fi <- function(fit) {
  if (is.null(fit) || !inherits(fit, "lavaan") || !lavInspect(fit, "converged")) {
    tibble::tibble(cfi = NA_real_, tli = NA_real_, rmsea = NA_real_, srmr = NA_real_)
  } else {
    tibble::tibble(
      cfi = fitMeasures(fit, "cfi"),
      tli = fitMeasures(fit, "tli"),
      rmsea = fitMeasures(fit, "rmsea"),
      srmr = fitMeasures(fit, "srmr")
    )
  }
}
fits_df <- dplyr::bind_rows(CFA = fi(fit_cfa),
                            ESEM = fi(fit_esem),
                            BIFACTOR = fi(fit_bif), .id = "Model")

fits_long <- fits_df |>
  tidyr::pivot_longer(cols = c(cfi,tli,rmsea,srmr), names_to = "Index", values_to = "Value")

p_fit <- ggplot(fits_long, aes(Model, Value, fill = Index)) +
  geom_col(position = position_dodge(width = 0.8), width = 0.7) +
  theme_minimal(base_size = 10) +
  labs(title = "Model fit comparison (key indices)", y = "Value", x = "")
ggsave("outputs/diagnostics/plot_fit_comparison.png", p_fit, width = 7, height = 4, dpi = 200,bg = "white")


New names:
• `cfi` -> `cfi...1`
• `cfi` -> `cfi...2`
• `cfi` -> `cfi...3`
New names:
• `tli` -> `tli...1`
• `tli` -> `tli...2`
• `tli` -> `tli...3`
New names:
• `rmsea` -> `rmsea...1`
• `rmsea` -> `rmsea...2`
• `rmsea` -> `rmsea...3`
New names:
• `srmr` -> `srmr...1`
• `srmr` -> `srmr...2`
• `srmr` -> `srmr...3`


In [37]:
sink("outputs/diagnostics/phase2_summary.txt")
cat("Phase 2 Summary\n")
cat("Chosen DASS model:", if (identical(fit_dass_final, fit_esem)) "ESEM" else if (identical(fit_dass_final, fit_cfa)) "3-factor CFA" else "Bifactor CFA", "\n")
cat("BRS model: one-factor CFA (WLSMV, theta)\n")
cat("Key fit (DASS final):\n")
print(fitMeasures(fit_dass_final, c("cfi","tli","rmsea","srmr","chisq","df")))
cat("\nKey fit (BRS):\n")
print(fitMeasures(fit_brs, c("cfi","tli","rmsea","srmr","chisq","df")))
sink()
